In [14]:
import torch
from torch_geometric.data import Data

In [15]:
data = torch.load("Graph/usu_graph.pth")

In [16]:
# Klasse 1 ist unterrepräsentiert
class_0_indices = torch.where(data.y == 0)[0]
class_1_indices = torch.where(data.y == 1)[0]

# Erzeugen synthetischer Datenpunkte für Klasse 1
num_synthetic_samples = len(class_0_indices) - len(class_1_indices)

# Schritt 1: Wähle zufällig einige Knoten aus der unterrepräsentierten Klasse
synthetic_indices = torch.randperm(len(class_1_indices))[:num_synthetic_samples]

# Schritt 2: Finde die Nachbarn für ausgewählte Knoten
neighbor_indices = []
for idx in synthetic_indices:
    neighbors = data.edge_index[1, data.edge_index[0] == idx]
    neighbor_indices.extend(neighbors.tolist())

# Schritt 3: Erstelle synthetische Datenpunkte basierend auf den Nachbarn
synthetic_data_points = []
for idx in synthetic_indices:
    neighbors = data.edge_index[1, data.edge_index[0] == idx]
    neighbor_features = data.x[neighbors]
    synthetic_point = torch.mean(neighbor_features, dim=0)  # Beispiel: Durchschnittliche Merkmale der Nachbarn
    synthetic_data_points.append(synthetic_point)

# Schritt 4: Integriere synthetische Daten in das Data-Objekt
synthetic_data = Data(
    x=torch.stack(synthetic_data_points),  # Features der synthetischen Daten
    edge_index=data.edge_index,       # Die Kantenstruktur bleibt erhalten
    y=torch.ones(len(synthetic_data_points), dtype=torch.long),  # Klasse 1
    train_mask=torch.ones(len(synthetic_data_points), dtype=torch.bool),  # Trainingsmasken für synthetische Daten
    val_mask=torch.zeros(len(synthetic_data_points), dtype=torch.bool),    # Validierungsmasken
    test_mask=torch.zeros(len(synthetic_data_points), dtype=torch.bool)    # Testmasken
)


In [17]:
synthetic_data

Data(x=[813, 25], edge_index=[2, 7132958], y=[813], train_mask=[813], val_mask=[813], test_mask=[813])

In [18]:
# Zusammenfügen des alten Datensatzes mit neuen Synthetischem Datensatz
data.x = torch.cat([data.x, synthetic_data.x], dim=0)
data.y = torch.cat([data.y, synthetic_data.y], dim=0)
data.train_mask = torch.cat([data.train_mask, synthetic_data.train_mask], dim=0)
data.val_mask = torch.cat([data.val_mask, synthetic_data.val_mask], dim=0)
data.test_mask = torch.cat([data.test_mask, synthetic_data.test_mask], dim=0)

In [19]:
torch.save(data, "Graph/syntetic_data.pth")